# Implementation of SimCLR with training the encoder and after the classification head

**The encoder can be pretrained using an unsupervised way. It is after trained using the SimCLR algorithm and the classification head is trained by supervised learning**

In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # disable GPU devices
os.environ["TFDS_DATA_DIR"] = os.path.expanduser("~/tensorflow_datasets")  # default location of tfds database
os.environ["KERAS_BACKEND"] = "tensorflow"

import keras
from keras import layers, models, regularizers
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

import tensorflow as tf
import tensorflow_datasets as tfds

from pathlib import Path

import matplotlib.pyplot as plt

# Turn off logging for TF
import logging
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
tf.get_logger().setLevel(logging.ERROR)

from dpmhm.datasets import preprocessing, feature, utils, transformer, query_parameters

2024-06-24 10:56:53.057681: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-24 10:56:53.061870: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-24 10:56:53.122351: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-24 10:56:54.094951: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
nb_elem_dirg=10000
nb_elem_paderborn= 10000
ds_train_size = nb_elem_dirg+nb_elem_paderborn

batch_size = 64
n_embedding  = 256 
kernel_size = (3,3) 
tau = 0.1
projection_dim = 128 

# Preprocessing on data

In [3]:
outdir = Path('/volatile/home/bm279471/tmp/meta_dataset')
os.makedirs(outdir, exist_ok=True)

# def pipeline(ds_name:str, *, split:str='all', channels:list=[], keys:list=None):
#     """Pipeline of preprocessing.

#     Parameters
#     ----------
#     ds_name
#         name of the dataset
#     split, optional
#         split to load, by default 'all'
#     channels, optional
#         channels to load, by default load all channels simultaneously
#     keys, optional
#         keys for the ramnification of labels

#     """
#     ds0 = tfds.load(ds_name, split=split)
#     if keys is None:
#         keys = query_parameters(ds_name)['keys'].keys()

#     compactor = transformer.DatasetCompactor(
#         ds0,
#         channels=channels, # select all channels simultaneously
#         keys=keys,
#         # resampling_rate=12000,
#         # split_channel=True,  # split multidimensional signals into 1d signals, incompatible with the pretrained VGGish model
#     )

#     _func = lambda x, sr: feature.spectral_features(
#         x, sr, 'spectrogram',
#         # n_mfcc=256,
#         time_window=0.025, hop_step=0.0125,
#         # n_fft=512,
#         normalize=False, to_db=True)[0]

#     extractor = transformer.FeatureExtractor(compactor.dataset, _func)
#     window = transformer.WindowSlider(extractor.dataset, window_size=(64,64), hop_size=(32,32))

#     labels = list(compactor.full_label_dict.keys())  # need the whole list of labels

#     return window.dataset, labels

# ds_list = ['CWRU', 'DIRG', 'Paderborn']
# ds_all = {}

# ds_all['CWRU'] = pipeline('CWRU')
# ds_all['DIRG'] = pipeline('DIRG', split='variation', channels=['A1'])
# ds_all['Paderborn'] = pipeline('Paderborn', split='healthy[:25%]+artificial[:25%]', channels=['vibration', 'current'])

# ds1, ds2, ds3 = ds_all['CWRU'][0], ds_all['DIRG'][0], ds_all['Paderborn'][0]
# lb1, lb2, lb3 = ds_all['CWRU'][1], ds_all['DIRG'][1], ds_all['Paderborn'][1]
# print(ds1.element_spec)
# print(ds2.element_spec)
# print(ds3.element_spec)

In [4]:
# ds2=ds2.take(nb_elem_dirg)
# ds3=ds3.take(nb_elem_paderborn)
# ds_train = ds2.concatenate(ds3)

# preproc_train = preprocessing.get_mapping_supervised(lb2+lb3)
# preproc_test = preprocessing.get_mapping_supervised(lb1)

# ds_train = utils.restore_shape(
#     ds_train.map(preproc_train, num_parallel_calls=tf.data.AUTOTUNE))
# ds_test = utils.restore_shape(
#     ds1.map(preproc_test, num_parallel_calls=tf.data.AUTOTUNE))

# eles = list(ds_train.take(10).as_numpy_iterator())
# input_shape = eles[0][0].shape

In [5]:
# ds_train = ds_train.map(lambda x,y: (tf.ensure_shape(x, input_shape), y), num_parallel_calls=tf.data.AUTOTUNE)
# ds_test = ds_test.map(lambda x,y: (tf.ensure_shape(x, input_shape), y), num_parallel_calls=tf.data.AUTOTUNE)

# split_train = {'train':0.8, 'val':0.2}
# ds_split_train = utils.split_dataset(ds_train, split_train)

# split_test = {'fine-tuning':0.1,'val':0.05, 'test':0.85}
# ds_split_test = utils.split_dataset(ds_test, split_test,labels=[i+1 for i in range(len(lb1))])

In [6]:
# import json

# ds_split_train['train'].save(str(outdir/'ds_train'))
# ds_split_train['val'].save(str(outdir/'ds_val'))
# ds_split_test['fine-tuning'].save(str(outdir/'ds_train_ft'))
# ds_split_test['val'].save(str(outdir/'ds_val_ft'))
# ds_split_test['test'].save(str(outdir/'ds_test_ft'))

# with open(outdir/'lb1.json', 'w') as fp:
#     json.dump(lb1,fp)
# with open(outdir/'lb2.json', 'w') as fp:
#     json.dump(lb2,fp)
# with open(outdir/'lb3.json', 'w') as fp:
#     json.dump(lb3,fp)

In [7]:
import json

ds_train = tf.data.Dataset.load(str(outdir/'ds_train'))
ds_val = tf.data.Dataset.load(str(outdir/'ds_val'))
ds_train_ft = tf.data.Dataset.load(str(outdir/'ds_train_ft'))
ds_val_ft = tf.data.Dataset.load(str(outdir/'ds_val_ft'))
ds_test_ft = tf.data.Dataset.load(str(outdir/'ds_test_ft'))

with open(outdir/'lb1.json', 'r') as fp:
    lb1 = list(json.load(fp))
with open(outdir/'lb2.json', 'r') as fp:
    lb2 = list(json.load(fp))
with open(outdir/'lb3.json', 'r') as fp:
    lb3 = list(json.load(fp))

ds_train_clr = ds_train.map(lambda x,l:(x,x)).shuffle(ds_train_size, reshuffle_each_iteration=False).cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_val_clr = ds_val.cache().batch(batch_size,drop_remainder=True)

ds_train_class = ds_train.shuffle(ds_train_size, reshuffle_each_iteration=False).cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)
ds_val_class = ds_val.cache().batch(batch_size,drop_remainder=True)

ds_test_size = utils.get_dataset_size(ds_train_ft)+utils.get_dataset_size(ds_val_ft)+utils.get_dataset_size(ds_test_ft)

ds_train_ft= ds_train_ft.shuffle(ds_test_size, reshuffle_each_iteration=False).cache().batch(batch_size,drop_remainder=True).prefetch(tf.data.AUTOTUNE)
ds_val_ft = ds_val_ft.cache().batch(batch_size,drop_remainder=True)
ds_test_ft=ds_test_ft.cache().batch(1)

2024-06-24 10:56:56.246292: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-06-24 10:56:56.246342: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: is230816
2024-06-24 10:56:56.246359: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: is230816
2024-06-24 10:56:56.246622: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:165] libcuda reported version is: 535.171.4
2024-06-24 10:56:56.246673: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:169] kernel reported version is: 535.171.4
2024-06-24 10:56:56.246686: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:248] kernel version seems to match DSO: 535.171.4
2024-06-24 10:56:56.743668: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RA

In [8]:
ds_test_size = utils.get_dataset_size(ds_train_ft)+utils.get_dataset_size(ds_val_ft)+utils.get_dataset_size(ds_test_ft)
eles = list(ds_train_clr.take(1).as_numpy_iterator())
input_shape = eles[0][0][0].shape
print(input_shape)

2024-06-24 10:57:00.163567: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-06-24 10:57:00.242999: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-06-24 10:57:03.455799: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


(64, 64, 3)


2024-06-24 10:57:04.867216: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-06-24 10:57:04.870779: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


# With an encoder from scratch

**Autoencoder**

Create the autoencoder

In [9]:
@keras.utils.register_keras_serializable()
class Autoencoder(models.Model):
    """Convolution Auto-Encoder stacks.


    Notes
    -----
    Shape (H,W) of the input tensor must be power of 2.
    """
    def __init__(self, input_shape, n_embedding,kernel_size):
        self.input_shape = input_shape
        activation = 'relu'
        padding = 'same'
        strides = (2,2)
        pool_size = (2,2)
        a_reg = 0. 

        super(Autoencoder, self).__init__()

        # Use more blocks and larger kernel size to get more smoothing in the reconstruction.
        input_layer= layers.Input(shape=input_shape, name='input_enc')

        layers_encoder = [
            # Block 1
            layers.Conv2D(32, kernel_size=kernel_size, activation=activation, padding=padding, name='conv1_enc'),
            layers.MaxPooling2D(pool_size=pool_size, strides=strides, name='pool1_enc'),
            layers.BatchNormalization(name='bn1_enc'), # by default axis=-1 for channel-last

            # Block 2
            layers.Conv2D(64, kernel_size=kernel_size, activation=activation, padding=padding, name='conv2_enc'),
            layers.MaxPooling2D(pool_size=pool_size, strides=strides, name='pool2_enc'),
            layers.BatchNormalization(name='bn2_enc'),

            # Block 3
            layers.Conv2D(128, kernel_size=kernel_size, activation=activation, padding=padding, name='conv3_enc'),
            layers.MaxPooling2D(pool_size=pool_size, strides=strides, name='pool3_enc'),
            layers.BatchNormalization(name='bn3_enc'),

            # Block fc
            layers.Flatten(name='flatten'),
            layers.Dense(n_embedding, activation=activation,activity_regularizer=regularizers.L1(a_reg), name='fc1_enc') if a_reg > 0
            else layers.Dense(n_embedding, activation=activation, name='fc1_enc')
        ]

        self.encoder = models.Sequential([input_layer] +layers_encoder, name='encoder')

        output_layer = layers.Input(shape=(n_embedding,), name='input_dec')
        layers_decoder = [
            # Block fc
            layers.Dense(128 * (input_shape[0] // 8) * (input_shape[1] // 8), activation=activation, activity_regularizer=regularizers.L1(a_reg), name='fc1_dec') if a_reg > 0 else layers.Dense(128 * (input_shape[0] // 8) * (input_shape[1] // 8), activation=activation, name='fc1_dec'),
            layers.Reshape((input_shape[0] // 8, input_shape[1] // 8, 128), name='reshape'),

            # Block 3
            layers.BatchNormalization(name='bn3_dec'),
            layers.UpSampling2D(strides, name='ups3_dec'),
            layers.Conv2DTranspose(64, kernel_size=kernel_size, activation=activation, padding=padding, name='tconv3_dec'),

            # Block 2
            layers.BatchNormalization(name='bn2_dec'),
            layers.UpSampling2D(strides, name='ups2_dec'),
            layers.Conv2DTranspose(32, kernel_size=kernel_size, activation=activation, padding=padding, name='tconv2_dec'),

            # Block 1
            layers.BatchNormalization(name='bn1_dec'),
            layers.UpSampling2D(strides, name='ups1_dec'),
            layers.Conv2DTranspose(input_shape[-1], kernel_size=kernel_size, activation=None, padding=padding, name='tconv1_dec'),
        ]

        self.decoder = models.Sequential([output_layer] + layers_decoder, name='decoder')

        self.autoencoder = models.Sequential([input_layer] + layers_encoder + layers_decoder, name='auto-encoder')

    def call(self, x):
        return self.decoder(self.encoder(x))

Compile and pretrain the autoencoder on unlabeled data under the form (img,img)

In [10]:
autoencoder = Autoencoder(input_shape,n_embedding,kernel_size)

# ds_train_autoencodeur = ds_split_train['train'].map(lambda x,l:(x,x))
# ds_val_autoencodeur = ds_split_train['val'].map(lambda x,l:(x,x)).batch(batch_size)

# ds_train_autoencodeur = ds_train_autoencodeur.shuffle(ds_train_size, reshuffle_each_iteration=True).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# autoencoder.compile(
#     optimizer=keras.optimizers.Adam(),
#     loss=keras.losses.MeanSquaredError(),
#     # metrics=['accuracy'],
# )

# history = autoencoder.fit(
#     ds_train_autoencodeur,
#     validation_data=ds_val_autoencodeur,
#     epochs=10,
#     callbacks=keras.callbacks.EarlyStopping(verbose=1, patience=3),
#     steps_per_epoch=int((0.8*ds_train_size) // batch_size)
# )

In [11]:
# autoencoder.save_weights('model_weights.weights.h5')

In [12]:
# autoencoder.load_weights('model_weights.weights.h5')

**Contrastive learning on the encoder**

Contrastive loss function

In [13]:
def contrastive_loss_fn(z_i, z_j, tau=0.5):
    z_i = tf.math.l2_normalize(z_i, axis=1)
    z_j = tf.math.l2_normalize(z_j, axis=1)

    # Compute the similarity matrix
    similarity_matrix = tf.matmul(z_i, z_j, transpose_b=True) / tau

    # Compute the positive similarity
    positive_similarity = tf.linalg.diag_part(similarity_matrix)

    # Compute the negative similarity
    negative_similarity = tf.linalg.set_diag(similarity_matrix, tf.zeros_like(tf.linalg.diag_part(similarity_matrix)))

    # Compute the numerator of the loss function
    numerator = tf.exp(positive_similarity)

    # Compute the denominator of the loss function
    denominator = tf.reduce_sum(tf.exp(negative_similarity), axis=1)

    # Compute the loss function
    loss = -tf.reduce_mean(tf.math.log(numerator / denominator))

    return loss

Create the SimCLR model

In [14]:
tf.config.experimental_run_functions_eagerly(True)

# Define the contrastive model with model-subclassing
class SimCLRModel(keras.Model):
    def __init__(self):
        super().__init__()

        self.tau = tau

        self.contrastive_augmenter = keras.Sequential([
            layers.RandomFlip("horizontal_and_vertical"),
            layers.RandomZoom(0.2),
            layers.RandomTranslation(height_factor=0.2, width_factor=0.2),
        ], name='Data_augmentation')
        
        self.encoder = autoencoder.encoder

        self.projection_head = keras.Sequential([
                layers.Dense(256, activation='relu'),
                layers.BatchNormalization(),
                layers.Dense(128, activation='relu'),
                layers.BatchNormalization(),
                layers.Dense(projection_dim),
            ], name='Projection_head')
        
        self.classification_head = keras.Sequential([
                layers.Input(shape=(128,)),
                layers.Dense(128, activation='relu'),
                layers.BatchNormalization(),
                layers.Dense(len(lb2+lb3)) #nb labels
            ], name='Classification_head')

    def compile(self, contrastive_optimizer, **kwargs):
        super().compile(**kwargs)

        self.contrastive_optimizer = contrastive_optimizer

        self.contrastive_loss_tracker = keras.metrics.Mean(name="c_loss")
        self.mean_cosine_similarity = keras.metrics.Mean(name="mean_cosine_similarity")

    @property
    def metrics(self):
        return [
            self.contrastive_loss_tracker,
            self.mean_cosine_similarity,
        ]

    def train_step(self, data):
        train_image, _ = data
        
        # Each set of unlabeled images is augmented separately
        augmented_image_1 = self.contrastive_augmenter(train_image, training=True)
        augmented_image_2 = self.contrastive_augmenter(train_image, training=True)
        
        with tf.GradientTape() as tape:
            # Extract features and compute projections for the first set of images
            features_1 = self.encoder(augmented_image_1, training=True)
            projections_1 = self.projection_head(features_1, training=True)
            
            # Extract features and compute projections for the second set of images
            features_2 = self.encoder(augmented_image_2, training=True)
            projections_2 = self.projection_head(features_2, training=True)
            
            # Compute contrastive loss
            contrastive_loss = contrastive_loss_fn(projections_1, projections_2, self.tau)
        
        # Compute gradients and apply updates for the contrastive loss
        gradients = tape.gradient(
            contrastive_loss,
            self.encoder.trainable_weights + self.projection_head.trainable_weights,
        )
        self.contrastive_optimizer.apply_gradients(
            zip(
                gradients,
                self.encoder.trainable_weights + self.projection_head.trainable_weights,
            )
        )
        self.contrastive_loss_tracker.update_state(contrastive_loss)
        
        cosine_similarity = tf.reduce_mean(keras.losses.cosine_similarity(projections_1, projections_2))
        self.mean_cosine_similarity.update_state(cosine_similarity)

        return {m.name: m.result() for m in self.metrics[:2]}

    def test_step(self, data):
        test_image, _ = data

        # Compute contrastive loss on the validation set
        augmented_image_1 = self.contrastive_augmenter(test_image, training=True)
        augmented_image_2 = self.contrastive_augmenter(test_image, training=True)

        features_1 = self.encoder(augmented_image_1, training=False)
        features_2 = self.encoder(augmented_image_2, training=False)

        projections_1 = self.projection_head(features_1, training=False)
        projections_2 = self.projection_head(features_2, training=False)
        
        contrastive_loss = contrastive_loss_fn(projections_1, projections_2, self.tau)
        self.contrastive_loss_tracker.update_state(contrastive_loss)

        cosine_similarity = tf.reduce_mean(keras.losses.cosine_similarity(projections_1, projections_2))
        self.mean_cosine_similarity.update_state(cosine_similarity)

        return {m.name: m.result() for m in self.metrics}

Compile and train SimCLR model

In [15]:
simclr_model = SimCLRModel()

simclr_model.compile(
    contrastive_optimizer=keras.optimizers.Adam(),
)

early_stopping = EarlyStopping(
    monitor='val_c_loss',  
    patience=3,       
    restore_best_weights=True, 
    mode='min'
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_c_loss', 
    factor=0.1,         
    patience=2, 
    mode='min'    
)

simclr_history = simclr_model.fit(
    ds_train_clr.repeat(), 
    epochs=10, 
    validation_data=ds_val_clr, 
    steps_per_epoch=int((0.8*ds_train_size) // batch_size),
    callbacks=[early_stopping, reduce_lr]
)

simclr_model.summary()

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 193s 766ms/step - c_loss: 0.0380 - mean_cosine_similarity: -0.8688 - val_c_loss: 1.8884 - val_mean_cosine_similarity: -0.9835 - learning_rate: 0.0010
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 195s 780ms/step - c_loss: -0.6790 - mean_cosine_similarity: -0.8833 - val_c_loss: -0.7224 - val_mean_cosine_similarity: -0.9078 - learning_rate: 0.0010
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 204s 818ms/step - c_loss: -1.4534 - mean_cosine_similarity: -0.8631 - val_c_loss: -0.2753 - val_mean_cosine_similarity: -0.9303 - learning_rate: 0.0010
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 203s 812ms/step - c_loss: -1.5327 - mean_cosine_similarity: -0.8659 - val_c_loss: -0.8274 - val_mean_cosine_similarity: -0.9151 - learning_rate: 0.0010
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 208s 832ms/step - c_loss: -1.8028 - mean_cosine_similarity: -0.8633 - val_c_loss: -0.5042 - val_mean_cosine_similarity: -0.9180 - learning_rate: 0.0010
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 20

Model: "sim_clr_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Data_augmentation (Sequential)  │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder (Sequential)            │ ?                      │     2,191,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Projection_head (Sequential)    │ ?                      │       116,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Classification_head             │ ?                      │        19,088 │
│ (Sequential)                    │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,327,376 (8.88 MB)

 Trainable params: 2,325,904 (8.87 MB)

 Non-trainable params: 1,472 (5.75 KB)

In [16]:
autoencoder.save_weights('simclr.weights.h5')
# autoencoder.load_weights('model_weights.weights.h5')

Train the classification head

In [17]:
# Freeze the encoder weights
simclr_model.encoder.trainable = False

classification_head= keras.Sequential([
                layers.Input(shape=(n_embedding,)),
                layers.Dense(128, activation='relu'),
                layers.BatchNormalization(),
                layers.Dense(len(lb2+lb3)) #nb labels
            ], name='Classification_head')

# Create a new classification model
encoder_output = simclr_model.encoder(simclr_model.encoder.layers[0].input)
classification_model_metadataset = keras.Model(
    inputs=simclr_model.encoder.layers[0].input,
    outputs=classification_head(encoder_output)
)
classification_model_metadataset.summary(show_trainable=True, expand_nested=True)

classification_model_metadataset.compile(
    optimizer=keras.optimizers.Adam(), 
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics=['accuracy'])

classification_history = classification_model_metadataset.fit(
    ds_train_class.repeat(), 
    epochs=5, 
    validation_data=ds_val_class, 
    steps_per_epoch=int((0.8*ds_train_size) // batch_size))

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_enc (InputLayer)      │ (None, 64, 64, 3)     │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ encoder (Sequential)        │ (None, 256)           │  2,191,552 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ conv1_enc (Conv2D)     │ (None, 64, 64, 32)    │        896 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ pool1_enc              │ (None, 32, 32, 32)    │          0 │   -   │
│ (MaxPooling2D)              │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ bn1_enc                │ (None, 32, 32, 32)    │        128 │   N   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ conv2_enc (Conv2D)     │ (None, 32, 32, 64)    │     18,496 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ pool2_enc              │ (None, 16, 16, 64)    │          0 │   -   │
│ (MaxPooling2D)              │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ bn2_enc                │ (None, 16, 16, 64)    │        256 │   N   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ conv3_enc (Conv2D)     │ (None, 16, 16, 128)   │     73,856 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ pool3_enc              │ (None, 8, 8, 128)     │          0 │   -   │
│ (MaxPooling2D)              │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ bn3_enc                │ (None, 8, 8, 128)     │        512 │   N   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ flatten (Flatten)      │ (None, 8192)          │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ fc1_enc (Dense)        │ (None, 256)           │  2,097,408 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ Classification_head         │ (None, 16)            │     35,472 │   Y   │
│ (Sequential)                │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ dense_5 (Dense)        │ (None, 128)           │     32,896 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ batch_normalization_3  │ (None, 128)           │        512 │   Y   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ dense_6 (Dense)        │ (None, 16)            │      2,064 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 2,227,024 (8.50 MB)

 Trainable params: 35,216 (137.56 KB)

 Non-trainable params: 2,191,808 (8.36 MB)

Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 68s 265ms/step - accuracy: 0.6923 - loss: 0.8810 - val_accuracy: 0.8034 - val_loss: 0.4648
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 72s 287ms/step - accuracy: 0.8234 - loss: 0.4397 - val_accuracy: 0.8327 - val_loss: 0.3948
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 68s 274ms/step - accuracy: 0.8629 - loss: 0.3568 - val_accuracy: 0.8705 - val_loss: 0.3299
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 67s 267ms/step - accuracy: 0.8767 - loss: 0.3143 - val_accuracy: 0.9027 - val_loss: 0.2709
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 70s 281ms/step - accuracy: 0.9080 - loss: 0.2548 - val_accuracy: 0.8944 - val_loss: 0.2478


Training of a new classification head on the test dataset

In [18]:
# Freeze the encoder weights
simclr_model.encoder.trainable = False

classification_head_ft= keras.Sequential([
    layers.Input(shape=(n_embedding,)),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(len(lb1)) #nb labels
], name='Classification_head')

# Create a new classification model
encoder_output = simclr_model.encoder(simclr_model.encoder.layers[0].input)
classification_model_cwru = keras.Model(
    inputs=simclr_model.encoder.layers[0].input,
    outputs=classification_head_ft(encoder_output)
)

early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=3,       
    restore_best_weights=True  
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.1,         
    patience=2          
)

classification_model_cwru.compile(
    optimizer=keras.optimizers.Adam(), 
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics=['accuracy'])

classification_history = classification_model_cwru.fit(
    ds_train_ft.repeat(), 
    epochs=15, 
    validation_data=ds_val_ft, 
    steps_per_epoch=int((0.1*ds_test_size) // batch_size),
    callbacks=[early_stopping, reduce_lr])

classification_model_cwru.summary(show_trainable=True, expand_nested=True)

Epoch 1/15


21/21 ━━━━━━━━━━━━━━━━━━━━ 7s 339ms/step - accuracy: 0.0852 - loss: 3.4843 - val_accuracy: 0.0911 - val_loss: 4.5564 - learning_rate: 0.0010
Epoch 2/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 6s 302ms/step - accuracy: 0.1780 - loss: 2.9036 - val_accuracy: 0.1654 - val_loss: 3.0430 - learning_rate: 0.0010
Epoch 3/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 6s 294ms/step - accuracy: 0.2008 - loss: 2.5816 - val_accuracy: 0.1745 - val_loss: 2.9112 - learning_rate: 0.0010
Epoch 4/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 6s 296ms/step - accuracy: 0.2950 - loss: 2.3681 - val_accuracy: 0.2005 - val_loss: 2.6415 - learning_rate: 0.0010
Epoch 5/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 7s 354ms/step - accuracy: 0.3738 - loss: 2.1431 - val_accuracy: 0.2799 - val_loss: 2.3493 - learning_rate: 0.0010
Epoch 6/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 6s 297ms/step - accuracy: 0.4066 - loss: 1.9750 - val_accuracy: 0.2708 - val_loss: 2.3701 - learning_rate: 0.0010
Epoch 7/15
21/21 ━━━━━━━━━━━━━━━━━━━━ 6s 300ms/step - accuracy: 0.4756 - loss: 1.8339 - val_accurac

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_enc (InputLayer)      │ (None, 64, 64, 3)     │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ encoder (Sequential)        │ (None, 256)           │  2,191,552 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ conv1_enc (Conv2D)     │ (None, 64, 64, 32)    │        896 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ pool1_enc              │ (None, 32, 32, 32)    │          0 │   -   │
│ (MaxPooling2D)              │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ bn1_enc                │ (None, 32, 32, 32)    │        128 │   N   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ conv2_enc (Conv2D)     │ (None, 32, 32, 64)    │     18,496 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ pool2_enc              │ (None, 16, 16, 64)    │          0 │   -   │
│ (MaxPooling2D)              │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ bn2_enc                │ (None, 16, 16, 64)    │        256 │   N   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ conv3_enc (Conv2D)     │ (None, 16, 16, 128)   │     73,856 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ pool3_enc              │ (None, 8, 8, 128)     │          0 │   -   │
│ (MaxPooling2D)              │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ bn3_enc                │ (None, 8, 8, 128)     │        512 │   N   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ flatten (Flatten)      │ (None, 8192)          │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ fc1_enc (Dense)        │ (None, 256)           │  2,097,408 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ Classification_head         │ (None, 29)            │     37,149 │   Y   │
│ (Sequential)                │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ dense_7 (Dense)        │ (None, 128)           │     32,896 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ batch_normalization_4  │ (None, 128)           │        512 │   Y   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ dense_8 (Dense)        │ (None, 29)            │      3,741 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 2,302,489 (8.78 MB)

 Trainable params: 36,893 (144.11 KB)

 Non-trainable params: 2,191,808 (8.36 MB)

 Optimizer params: 73,788 (288.24 KB)

Fine-tuning

In [19]:
# Unfreeze the encoder weights
simclr_model.encoder.trainable = True

classification_model_cwru.compile(
    optimizer=keras.optimizers.Adam(), 
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics=['accuracy'])

classification_history = classification_model_cwru.fit(
    ds_train_ft.repeat(), 
    epochs=20, 
    validation_data=ds_val_ft, 
    steps_per_epoch=int((0.1*ds_test_size) // batch_size))

classification_model_cwru.summary(show_trainable=True, expand_nested=True)

Epoch 1/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 424ms/step - accuracy: 0.2201 - loss: 3.0964 - val_accuracy: 0.1133 - val_loss: 4.2714
Epoch 2/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 421ms/step - accuracy: 0.5908 - loss: 1.4173 - val_accuracy: 0.1380 - val_loss: 3.6293
Epoch 3/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 411ms/step - accuracy: 0.8164 - loss: 0.8169 - val_accuracy: 0.1484 - val_loss: 3.4192
Epoch 4/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 406ms/step - accuracy: 0.9411 - loss: 0.4586 - val_accuracy: 0.1641 - val_loss: 3.1972
Epoch 5/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 399ms/step - accuracy: 0.9646 - loss: 0.2943 - val_accuracy: 0.1758 - val_loss: 2.9964
Epoch 6/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 402ms/step - accuracy: 0.9883 - loss: 0.1637 - val_accuracy: 0.2018 - val_loss: 2.6901
Epoch 7/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 416ms/step - accuracy: 0.9956 - loss: 0.0999 - val_accuracy: 0.2578 - val_loss: 2.4932
Epoch 8/20
21/21 ━━━━━━━━━━━━━━━━━━━━ 9s 410ms/step - accuracy: 0.9953 - loss: 0.0743 - val_accuracy: 0.

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_enc (InputLayer)      │ (None, 64, 64, 3)     │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ encoder (Sequential)        │ (None, 256)           │  2,191,552 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ conv1_enc (Conv2D)     │ (None, 64, 64, 32)    │        896 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ pool1_enc              │ (None, 32, 32, 32)    │          0 │   -   │
│ (MaxPooling2D)              │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ bn1_enc                │ (None, 32, 32, 32)    │        128 │   Y   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ conv2_enc (Conv2D)     │ (None, 32, 32, 64)    │     18,496 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ pool2_enc              │ (None, 16, 16, 64)    │          0 │   -   │
│ (MaxPooling2D)              │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ bn2_enc                │ (None, 16, 16, 64)    │        256 │   Y   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ conv3_enc (Conv2D)     │ (None, 16, 16, 128)   │     73,856 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ pool3_enc              │ (None, 8, 8, 128)     │          0 │   -   │
│ (MaxPooling2D)              │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ bn3_enc                │ (None, 8, 8, 128)     │        512 │   Y   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ flatten (Flatten)      │ (None, 8192)          │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ fc1_enc (Dense)        │ (None, 256)           │  2,097,408 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ Classification_head         │ (None, 29)            │     37,149 │   Y   │
│ (Sequential)                │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ dense_7 (Dense)        │ (None, 128)           │     32,896 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ batch_normalization_4  │ (None, 128)           │        512 │   Y   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│    └ dense_8 (Dense)        │ (None, 29)            │      3,741 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 6,684,697 (25.50 MB)

 Trainable params: 2,227,997 (8.50 MB)

 Non-trainable params: 704 (2.75 KB)

 Optimizer params: 4,455,996 (17.00 MB)

Evaluate the model 

In [20]:
evaluation = classification_model_cwru.evaluate(ds_test_ft)

# Print the evaluation metrics
print("Evaluation Loss:", evaluation[0])
print("Evaluation Accuracy: {:.2f}%".format(evaluation[1]*100))

    1/13873 ━━━━━━━━━━━━━━━━━━━━ 7:50 34ms/step - accuracy: 1.0000 - loss: 0.0196

13873/13873 ━━━━━━━━━━━━━━━━━━━━ 289s 21ms/step - accuracy: 0.9802 - loss: 0.1061
Evaluation Loss: 0.08104901015758514
Evaluation Accuracy: 98.77%
